Test for one course  price , syllabus , about course , course faculty , language

In [12]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time

# === Input: One Course URL ===
course_link = "https://www.eduonix.com/devops-labs-14-real-devops-projects"

# === Setup Headless Chrome ===
chrome_options = Options()
chrome_options.add_argument("--headless=new")
chrome_options.add_argument("--disable-gpu")
driver = webdriver.Chrome(options=chrome_options)

# === Utility: Check if text is a question ===
def is_question(text):
    question_starters = ['why', 'how', 'what', 'when', 'can', 'do', 'does', 'is', 'are', 'which']
    text = text.strip().lower()
    return text.endswith('?') or any(text.startswith(q) for q in question_starters)

# === Load and Parse Page ===
driver.get(course_link)
time.sleep(3)
soup = BeautifulSoup(driver.page_source, 'html.parser')

# === Extract Course Name ===
course_name_tag = soup.find('h1', class_='productTopHeading')
course_name = course_name_tag.get_text(strip=True) if course_name_tag else "Unknown Course Name"

# === Extract Language ===
language = "Unknown"
lang_spans = soup.find_all('span', class_='divInfo')
for span in lang_spans:
    if 'fa-globe' in str(span):
        language = span.get_text(strip=True)
        break

# === Extract Instructor ===
faculty = "Unknown"
instructor_block = soup.find('b', string="Instructor:")
if instructor_block and instructor_block.next_sibling:
    faculty_tag = instructor_block.find_next('a')
    if faculty_tag:
        faculty = faculty_tag.get_text(strip=True)

# === Extract Price ===
price_tag = soup.find('span', id='mnumber')
price = price_tag.get_text(strip=True) if price_tag else "Unknown Price"

# === Extract About Course ===
about_course = "Not Available"
about_div = soup.find('div', class_='viewMoreDescription')
if about_div:
    about_course = '\n'.join(p.get_text(strip=True) for p in about_div.find_all('p') if p.get_text(strip=True))

# === Extract Syllabus ===
# === Extract All Course Titles ===
course_titles = []
for btn in soup.find_all('button', class_='edHead'):
    title_tag = btn.find('span', class_='edCardTitle')
    if title_tag:
        title = title_tag.get_text(strip=True)
        if title:
            course_titles.append(title)

# === Extract All Syllabus Content ===
syllabus_blocks = soup.find_all(['div', 'span', 'h4'])

syllabus_content = ""
current_course_index = -1

for block in syllabus_blocks:
    if block.name == 'span' and 'edCardTitle' in block.get('class', []):
        current_course_index += 1
        syllabus_content += f"\n🎓 {course_titles[current_course_index]}\n"

    elif block.name == 'h4' and 'curriculumName' in block.get('class', []):
        module_title_elem = block
        module_num_elem = block.find_previous('div', class_='moduleHeading')
        module_title = module_title_elem.get_text(strip=True)
        module_num = module_num_elem.get_text(strip=True) if module_num_elem else ''
        full_module_title = f"{module_num} {module_title}".strip()
        syllabus_content += f"\n🔸 {full_module_title}\n"

    elif block.name == 'span' and 'secInfo' in block.get('class', []):
        b_tag = block.find('b')
        if b_tag:
            title = b_tag.get_text(strip=True)
            if 'frequently asked' in title.lower() or 'faq' in title.lower() or is_question(title):
                continue
            syllabus_content += f"\n📘 {title.strip()}\n"

    elif block.name == 'div' and 'syllbusMain' in block.get('class', []):
        desc_div = block.find('div', class_='syllbusDesc')
        if desc_div:
            for btn in desc_div.find_all('button'):
                btn.decompose()
            lesson_text = desc_div.get_text(strip=True)
            if lesson_text:
                syllabus_content += f"- {lesson_text.strip()}\n"


driver.quit()

# === Final Output ===
if not syllabus_content.strip():
    syllabus_content = "No Syllabus Found"

# === Print Results ===
print(f"📗 Course Name: {course_name}")
print(f"🌐 Language: {language}")
print(f"👨‍🏫 Course Faculty: {faculty}")
print(f"💰 Price: {price}")
print(f"\n📝 About Course:\n{about_course}\n")
print(syllabus_content.strip())


📗 Course Name: DevOps Labs: 14 Real DevOps Projects
🌐 Language: English (US)
👨‍🏫 Course Faculty: Brian Su
💰 Price: ₹995.00

📝 About Course:
DevOps Labs: 14 Real DevOps Projectsis your hands-on path to mastering the most in-demand DevOps skills. Whether you're an aspiring DevOps engineer, a developer aiming to understand infrastructure, or someone preparing for technical job interviews, this course equips you with practical experience across real-world scenarios.
Containerization with DockerBuild, run, and manage containerized applications.
Orchestration using KubernetesDeploy and scale apps using Kubernetes clusters.
Version Control with Git, GitHub & GitLabCollaborate, manage source code, and automate pipelines.
CI/CD PipelinesImplement Continuous Integration and Continuous Delivery workflows from scratch.
Infrastructure as CodeLearn to define, manage, and provision infrastructure efficiently.
Job Interview ReadinessTackle interview questions with confidence using project-based insigh

EXTRACTED  FOR ALL COURSES WEB SCRAPPER  # IMP

In [20]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time

# === Input Excel file ===
input_excel_path = "C:\\Users\\taslim.siddiqui\\Downloads\\eduonix_link2.xlsx"
df_links = pd.read_excel(input_excel_path)
course_links = df_links['Course Link'].dropna().tolist()

# === Setup Chrome ===
chrome_options = Options()
chrome_options.add_argument("--headless=new")
chrome_options.add_argument("--disable-gpu")
driver = webdriver.Chrome(options=chrome_options)

# === Utility ===
def is_question(text):
    question_starters = ['why', 'how', 'what', 'when', 'can', 'do', 'does', 'is', 'are', 'which']
    text = text.strip().lower()
    return text.endswith('?') or any(text.startswith(q) for q in question_starters)

# === Scrape each course ===
all_data = []

for i, link in enumerate(course_links):
    print(f"Scraping ({i+1}/{len(course_links)}): {link}")
    try:
        driver.get(link)
        time.sleep(3)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # --- Course Name ---
        course_name_tag = soup.find('h1', class_='productTopHeading')
        course_name = course_name_tag.get_text(strip=True) if course_name_tag else "Unknown Course Name"

        # --- Language ---
        language = "Not found"
        lang_spans = soup.find_all('span', class_='divInfo')
        for span in lang_spans:
            if 'fa-globe' in str(span):
                language = span.get_text(strip=True)
                break

        # --- Course Faculty ---
        faculty = "Not found"
        instructor_tag = soup.find('a', href=lambda x: x and '/i/' in x)
        if instructor_tag:
            faculty = instructor_tag.get_text(strip=True)

        # --- Price ---
        price = "Not found"
        price_tag = soup.find('span', id='mnumber')
        if price_tag:
            price_text = price_tag.get_text(strip=True)
            price = price_text.replace('\xa0', ' ').replace('₹', ' ').strip()
        else:
            strike_tag = soup.find('strike', class_='original_price')
            if strike_tag:
                price_text = strike_tag.get_text(strip=True)
                price = price_text.replace('\xa0', ' ').replace('₹', ' ').strip()
            else:
                cut_price_tag = soup.find('div', class_='cutPrice')
                if cut_price_tag:
                    price_text = cut_price_tag.get_text(strip=True)
                    price = price_text.replace('\xa0', ' ').replace('₹', '').strip()

        # --- Certificate ---
        certificate = "No"
        certificate_div = soup.find('div', class_='courseCertificate')
        if certificate_div:
            certificate = "Yes"

        # --- About Course ---
        about_course = "Not Available"
        about_div = soup.find('div', class_='viewMoreDescription')
        if about_div:
            about_course = '\n'.join(
                p.get_text(strip=True) for p in about_div.find_all('p') if p.get_text(strip=True)
            )

        # --- Course Syllabus ---
        course_titles = []
        for btn in soup.find_all('button', class_='edHead'):
            title_tag = btn.find('span', class_='edCardTitle')
            if title_tag:
                title = title_tag.get_text(strip=True)
                if title:
                    course_titles.append(title)

        syllabus_blocks = soup.find_all(['div', 'span', 'h4'])

        syllabus_content = ""
        current_course_index = -1

        for block in syllabus_blocks:
            if block.name == 'span' and 'edCardTitle' in block.get('class', []):
                current_course_index += 1
                if current_course_index < len(course_titles):
                    syllabus_content += f"\n🎓 {course_titles[current_course_index]}\n"

            elif block.name == 'h4' and 'curriculumName' in block.get('class', []):
                module_title_elem = block
                module_num_elem = block.find_previous('div', class_='moduleHeading')
                module_title = module_title_elem.get_text(strip=True)
                module_num = module_num_elem.get_text(strip=True) if module_num_elem else ''
                full_module_title = f"{module_num} {module_title}".strip()
                syllabus_content += f"\n🔸 {full_module_title}\n"

            elif block.name == 'span' and 'secInfo' in block.get('class', []):
                b_tag = block.find('b')
                if b_tag:
                    title = b_tag.get_text(strip=True)
                    if 'frequently asked' in title.lower() or 'faq' in title.lower() or is_question(title):
                        continue
                    syllabus_content += f"\n📘 {title.strip()}\n"

            elif block.name == 'div' and 'syllbusMain' in block.get('class', []):
                desc_div = block.find('div', class_='syllbusDesc')
                if desc_div:
                    for btn in desc_div.find_all('button'):
                        btn.decompose()
                    lesson_text = desc_div.get_text(strip=True)
                    if lesson_text:
                        syllabus_content += f"- {lesson_text.strip()}\n"

        # === Save course data ===
        course_data = {
            'Course Name': course_name,
            'Course Link': link,
            'Language': language,
            'Course Faculty': faculty,
            'Price': price,
            'Certificate': certificate,
            'About Course': about_course,
            'Course Syllabus': syllabus_content.strip()
        }

    except Exception as e:
        print(f"❌ Error scraping {link}: {e}")
        course_data = {
            'Course Name': '',
            'Course Link': link,
            'Language': '',
            'Course Faculty': '',
            'Price': '',
            'Certificate': '',
            'About Course': '',
            'Course Syllabus': ''
        }

    all_data.append(course_data)
    time.sleep(1)

# === Save to Excel ===
driver.quit()
df_out = pd.DataFrame(all_data)
output_path = "C:\\Users\\taslim.siddiqui\\Downloads\\eduonix_scraped_output.xlsx"
df_out.to_excel(output_path, index=False)
print("✅ All courses saved to:", output_path)


Scraping (1/9): https://www.eduonix.com/gpt-tools-for-marketing-edegree
Scraping (2/9): https://www.eduonix.com/management-skills-team-leadership-skills-masterclass-2019
Scraping (3/9): https://www.eduonix.com/gpt-tools-for-marketing-edegree
Scraping (4/9): https://www.eduonix.com/nail-node-js-amp-javascript-job-interviews 
Scraping (5/9): https://www.eduonix.com/complete-java-9-masterclass-beginner-to-expert
Scraping (6/9): https://www.eduonix.com/lifetime-ethical-hacking-and-cyber-security-membership
Scraping (7/9): https://www.eduonix.com/lifetime-data-science-machine-learning-membership
Scraping (8/9): https://www.eduonix.com/mastering-modern-workplace-essentials
Scraping (9/9): https://www.eduonix.com/lifetime-web-development-membership
✅ All courses saved to: C:\Users\taslim.siddiqui\Downloads\eduonix_scraped_output.xlsx


Extracted course syllbaus for one link

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time

course_link = "https://www.eduonix.com/lifetime-learning-access?coupon_code=LF199IND"
# Set up headless browser
chrome_options = Options()
chrome_options.add_argument("--headless=new")
chrome_options.add_argument("--disable-gpu")
driver = webdriver.Chrome(options=chrome_options)

# Helper to detect question-style section headers (FAQs
def is_question(text):
    question_starters = ['why', 'how', 'what', 'when', 'can', 'do', 'does', 'is', 'are', 'which']
    text = text.strip().lower()
    return text.endswith('?') or any(text.startswith(q) for q in question_starters)

# Load the page
driver.get(course_link)
time.sleep(3)
soup = BeautifulSoup(driver.page_source, 'html.parser')

# === Extract All Course Titles ===
course_titles = []
for btn in soup.find_all('button', class_='edHead'):
    title_tag = btn.find('span', class_='edCardTitle')
    if title_tag:
        title = title_tag.get_text(strip=True)
        if title:
            course_titles.append(title)

# === Extract All Syllabus Content ===
syllabus_blocks = soup.find_all(['div', 'span', 'h4'])

syllabus_content = ""
current_course_index = -1

for block in syllabus_blocks:
    if block.name == 'span' and 'edCardTitle' in block.get('class', []):
        current_course_index += 1
        syllabus_content += f"\n🎓 {course_titles[current_course_index]}\n"

    elif block.name == 'h4' and 'curriculumName' in block.get('class', []):
        module_title_elem = block
        module_num_elem = block.find_previous('div', class_='moduleHeading')
        module_title = module_title_elem.get_text(strip=True)
        module_num = module_num_elem.get_text(strip=True) if module_num_elem else ''
        full_module_title = f"{module_num} {module_title}".strip()
        syllabus_content += f"\n🔸 {full_module_title}\n"

    elif block.name == 'span' and 'secInfo' in block.get('class', []):
        b_tag = block.find('b')
        if b_tag:
            title = b_tag.get_text(strip=True)
            if 'frequently asked' in title.lower() or 'faq' in title.lower() or is_question(title):
                continue
            syllabus_content += f"\n📘 {title.strip()}\n"

    elif block.name == 'div' and 'syllbusMain' in block.get('class', []):
        desc_div = block.find('div', class_='syllbusDesc')
        if desc_div:
            for btn in desc_div.find_all('button'):
                btn.decompose()
            lesson_text = desc_div.get_text(strip=True)
            if lesson_text:
                syllabus_content += f"- {lesson_text.strip()}\n"

driver.quit()

# Save combined syllabus + link
all_data = [{
    'Course Link': course_link,
    'Syllabus': syllabus_content.strip()
}]

# Write to Excel
df = pd.DataFrame(all_data)
df.to_excel("C:\\Users\\taslim.siddiqui\\Downloads\\eduonix_syllabus_output.xlsx", index=False)

# Optional: print result to console
for entry in all_data:
    print("Course Link:", entry['Course Link'])
    print(entry['Syllabus'])
    print("\n" + "="*60 + "\n")


Extracted course syllabus only  for all links

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time

# Load Excel with course links
input_path = "C:\\Users\\taslim.siddiqui\\Downloads\\eduonix_link1.xlsx"
df_links = pd.read_excel(input_path)

# Validate column
if 'Course Link' not in df_links.columns:
    raise ValueError("Excel must contain a 'Course Link' column.")

# Headless browser setup
chrome_options = Options()
chrome_options.add_argument("--headless=new")
chrome_options.add_argument("--disable-gpu")

# Detect FAQ-like or question-style content
def is_question(text):
    question_starters = ['why', 'how', 'what', 'when', 'can', 'do', 'does', 'is', 'are', 'which']
    text = text.strip().lower()
    return text.endswith('?') or any(text.startswith(q) for q in question_starters)

# Loop over all links
all_data = []

for idx, row in df_links.iterrows():
    course_link = row['Course Link']
    print(f"⏳ Scraping ({idx + 1}/{len(df_links)}): {course_link}")
    
    driver = webdriver.Chrome(options=chrome_options)
    try:
        driver.get(course_link)
        time.sleep(3)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        syllabus_content = ""
        course_titles = [span.get_text(strip=True) for span in soup.find_all('span', class_='edCardTitle')]

        current_course_index = -1
        blocks = soup.find_all(['div', 'span', 'h4'])

        for block in blocks:
            # New Course Detected
            if block.name == 'span' and 'edCardTitle' in block.get('class', []):
                current_course_index += 1
                if current_course_index < len(course_titles):
                    syllabus_content += f"\n🎓 {course_titles[current_course_index]}\n"

            # Module title
            elif block.name == 'h4' and 'curriculumName' in block.get('class', []):
                module_title = block.get_text(strip=True)
                module_num_elem = block.find_previous('div', class_='moduleHeading')
                module_num = module_num_elem.get_text(strip=True) if module_num_elem else ''
                full_module_title = f"{module_num} {module_title}".strip()
                syllabus_content += f"\n🔸 {full_module_title}\n"

            # Section title
            elif block.name == 'span' and 'secInfo' in block.get('class', []):
                b_tag = block.find('b')
                if b_tag:
                    title = b_tag.get_text(strip=True)
                    if 'frequently asked' in title.lower() or 'faq' in title.lower() or is_question(title):
                        continue
                    syllabus_content += f"\n📘 {title}\n"

            # Lesson content
            elif block.name == 'div' and 'syllbusMain' in block.get('class', []):
                desc_div = block.find('div', class_='syllbusDesc')
                if desc_div:
                    for btn in desc_div.find_all('button'):
                        btn.decompose()
                    lesson_text = desc_div.get_text(strip=True)
                    if lesson_text:
                        syllabus_content += f"- {lesson_text}\n"

        all_data.append({
            'Course Link': course_link,
            'Syllabus': syllabus_content.strip()
        })

    except Exception as e:
        print(f"❌ Error scraping {course_link}: {e}")
        all_data.append({'Course Link': course_link, 'Syllabus': ''})

    finally:
        driver.quit()

# Save output to Excel
output_path = "C:\\Users\\taslim.siddiqui\\Downloads\\eduonix_syllabus_output.xlsx"
pd.DataFrame(all_data).to_excel(output_path, index=False)

print("\n✅ Scraping complete. Output saved to:", output_path)


⏳ Scraping (1/3): https://www.eduonix.com/lifetime-ethical-hacking-and-cyber-security-membership
⏳ Scraping (2/3): https://www.eduonix.com/lifetime-data-science-machine-learning-membership
⏳ Scraping (3/3): https://www.eduonix.com/lifetime-learning-access?coupon_code=LF199IND

✅ Scraping complete. Output saved to: C:\Users\taslim.siddiqui\Downloads\eduonix_syllabus_output.xlsx
